# Imports

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.edge.service import Service 
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

# Scraping

In [ ]:
companies = ['pwc', 'meta', 'google', 'infor', 'caustaza', 'sofrecom-tunisie', 'sofrecom']

## From linkedin

In [ ]:
def scrape_from_linkedin(company='caustaza', edgedriver_path = r'C:\Users\G5\Downloads\edgedriver_win64\msedgedriver.exe', email='mohamedomarsohnoon@gmail.com', password='', MAX_COUNT=20):
    # Set up Edge options 
    edge_options = Options()
    edge_options.use_chromium = True  # Ensure you're using the Chromium version of Edge
    edge_options.add_argument('--no-sandbox')

    # Set up the Edge service
    edge_service = Service(executable_path=edgedriver_path)
    
    # Initialize WebDriver
    driver = webdriver.Edge(service=edge_service, options=edge_options)

    # Open a website
    url = 'https://www.linkedin.com/company/'+company+'/posts/?feedView=all'
    driver.get(url)
    
    #sign in
    email_field = driver.find_element(by=By.ID, value='username')
    email_field.send_keys(email)
    password_field = driver.find_element(by=By.ID, value='password')
    password_field.send_keys(password)
    submit_button = driver.find_element(by=By.CSS_SELECTOR, value='.btn__primary--large.from__button--floating')
    submit_button.click()

    time.sleep(10)

    #load more posts
    count = 0
    while(1):
        try:
            scroll_button = driver.find_element(By.CSS_SELECTOR, '.artdeco-button.artdeco-button--muted.artdeco-button--1.artdeco-button--full.artdeco-button--secondary.ember-view.scaffold-finite-scroll__load-button')
            for i in range(5):
                try:
                    scroll_button.click()
                    count +=1 
                except(StaleElementReferenceException):
                    pass
            time.sleep(5)
        except(NoSuchElementException):
            break
        if (count == MAX_COUNT): 
            break

    container = driver.find_element(by=By.CSS_SELECTOR, value='.scaffold-finite-scroll__content')
    posts = container.find_elements(By.CSS_SELECTOR, '.ember-view.occludable-update')
    print(f'Number of posts: {len(posts)}'

    #scrape comments
    all_comments = []
    for post in tqdm(posts):
        buttons = post.find_elements(by=By.TAG_NAME, value='button')
        buttons_text = [button.text for button in buttons]
        comment_button_index = None
        for i in range(len(buttons)):
            if(buttons_text[i] == 'Comment'): 
                comment_button_index = i
                break
        if(comment_button_index != None): 
            time.sleep(3)
            comment_button = buttons[comment_button_index]
            for i in range(5):
                try: 
                    comment_button.click()
                    comments_section = post.find_element(By.CSS_SELECTOR, '.update-v2-social-activity')
                    try:
                        comments = comments_section.find_elements(By.CSS_SELECTOR, '.comments-comment-item-content-body.break-words')
                        comments_text = [comment.text for comment in comments]
                        all_comments.extend(comments_text)
                    except(NoSuchElementException):
                        pass
                    break
                except(ElementClickInterceptedException):
                    time.sleep(1)
    print(f'Comments count: {len(all_comments}')
    #save comments to a dataframe
    df = pd.DataFrame({'comment':all_comments})
    df.to_csv(f'data/{company}_linkedin_comments.csv', index=False)
    print('Comments saved successfully!')
    print(df.sample(10))
    driver.close()

In [ ]:
def scrape_all_from_linkedin(companies=companies):
    for company in tqdm(companies):
        scrape_from_linkedin(company=company)

'.x1n2onr6.x1ja2u2z.x78zum5.x2lah0s.xl56j7k.x6s0dn4.xozqiw3.x1q0g3np.xi112ho.x17zwfj4.x585lrc.x1403ito.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.xn6708d.x1ye3gou.xtvsq51.x1fq8qgq'

## From Facebook

In [93]:
def scrape_from_facebook(company='caustaza', edgedriver_path = r'C:\Users\G5\Downloads\edgedriver_win64\msedgedriver.exe', email='mohamedomarsohnoon@gmail.com', password='', N=5):
    # Set up Edge options 
    edge_options = Options()
    edge_options.use_chromium = True  # Ensure you're using the Chromium version of Edge
    edge_options.add_argument('--no-sandbox')
    
    # Set up the Edge service
    edge_service = Service(executable_path=edgedriver_path)
        
    # Initialize WebDriver
    driver = webdriver.Edge(service=edge_service, options=edge_options)
    
    # Open a website
    url = 'https://www.facebook.com/'+company
    driver.get(url)
    time.sleep(5)
    
    # Sign in
    email_tag = driver.find_element(By.CSS_SELECTOR, 'input.x1i10hfl xggy1nq x1s07b3s x1kdt53j x1a2a7pz xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x9f619 xzsf02u x1uxerd5 x1fcty0u x132q4wb x1a8lsjc x1pi30zi x1swvt13 x9desvi xh8yej3 x15h3p50 x10emqs4'.replace(' ', '.'))
    email_tag.send_keys(email)
    password_ = driver.find_elements(By.CSS_SELECTOR, 'input.x1i10hfl xggy1nq x1s07b3s x1kdt53j x1a2a7pz xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x9f619 xzsf02u x1uxerd5 x1fcty0u x132q4wb x1a8lsjc x1pi30zi x1swvt13 x9desvi xh8yej3 x15h3p50 x10emqs4'.replace(' ', '.'))[1]
    password_.send_keys(password)
    button = driver.find_element(By.CSS_SELECTOR, '.x1n2onr6 x1ja2u2z x78zum5 x2lah0s xl56j7k x6s0dn4 xozqiw3 x1q0g3np xi112ho x17zwfj4 x585lrc x1403ito x972fbf xcfux6l x1qhh985 xm0m39n x9f619 xn6708d x1ye3gou xtvsq51 x1fq8qgq'.replace(' ', '.'))
    button.click()
    print("logging in...")
    time.sleep(25)

    # Scrape
    comments=[]
    for i in range(N):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
    buttons = driver.find_elements(By.CSS_SELECTOR, '.x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm xi81zsa'.replace(' ', '.'))
    comment_buttons = [e for e in buttons if 'comment' in e.text]
    print(len(comment_buttons))
    for comment_button in comment_buttons:
        for i in range(5):
            try:
                comment_button.click()
                time.sleep(3)
                # Switch to the comments frame
                comments_elements = driver.find_elements(By.CSS_SELECTOR, '.x1lliihq.xjkvuk6.x1iorvi4')
                print(len(comments_elements))
                comments.extend([e.text for e in comments_elements])
                exit = driver.find_element(By.CSS_SELECTOR, '.x1i10hfl x1ejq31n xd10rxx x1sy0etr x17r0tee x1ypdohk xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r x16tdsg8 x1hl2dhg xggy1nq x87ps6o x1lku1pv x1a2a7pz x6s0dn4 xzolkzo x12go9s9 x1rnf11y xprq8jg x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x78zum5 xl56j7k xexx8yu x4uap5 x18d9i69 xkhd6sd x1n2onr6 xc9qbxq x14qfxbe x1qhmfi1'.replace(' ', '.'))
                exit.click()
                break
            except(ElementClickInterceptedException, StaleElementReferenceException):
                time.sleep(2)
    
    print(f'Comments count: {len(comments)}')
    #save comments to a dataframe
    df = pd.DataFrame({'comment':comments})
    df.to_csv(f'data/{company}_facebook_comments.csv', index=False)
    print('Comments saved successfully!')
    print(df.sample(10))
    driver.close()

In [94]:
scrape_from_facebook(company='microsoft', password='')

logging in...
21
37
37
35
36
36
35
36
35
33
33
32
31
31
31
30
31
Comments count: 539
Comments saved successfully!
                                               comment
494  I wish the heck( I would use a much stronger w...
199  @MicrosoftHelps can someone PLEASE call me bac...
258  Very informative as always. Have a nice rest o...
228  I was on support today for more than seven hou...
78   Microsoft keeps trying to send me a Teams Surv...
459  @MicrosoftHelps can someone PLEASE call me bac...
64   Very informative as always. Have a nice rest o...
154  Very informative as always. Have a nice rest o...
108                                                   
111  Technical question: How much did the photograp...


In [ ]:
def scrape_all_from_facebook(companies=companies):
    for company in tqdm(companies):
        scrape_from_facebook(company=company)